In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
import pandas as pd
import urllib.request
import os

In [7]:
URL = 'https://comic.naver.com/webtoon/weekday'
html = requests.get(URL).text # html 문서 전체를 긁어서 출력, .text는 태그 제외하고 text만 출력되게 함
soup = BeautifulSoup(html, 'html.parser')

title = soup.find_all('a', {'class' : 'title'}) # a태그에서 class='title'인 html소스를 찾아 할당
id_list = []
title_num_list = []
link_list = []
title_list = []
author_list = []
day_list = []
genre_list = []
story_list = []
imgSrc_list = []
num = 0

driver = webdriver.Chrome('C:/Users/Administrator/Desktop/JavaSpring/Webtoon_recommender/Webtoon_recommender/python_webtoon_crawler/chromedriver.exe')
driver.get(URL)

for i in range(len(title)):
    sleep(0.5)

    page = driver.find_elements_by_class_name('title')
    page[i].click() #월요일 첫 번째 웹툰부터 순서대로 클릭

    sleep(0.5)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser') # 이동한 페이지 주소 파싱

    day = soup.find_all('ul', {'class' : 'category_tab'})
    day = day[0].find('li', {'class' : 'on'}).text[0:1]

    t = title[i].text
    if (t in title_list):  # 한 작품이 요일 두 개 이상이면 요일만 추가
        day_list[title_list.index(t)] += ', ' + day
        driver.back()
        continue
    
    link = soup.find_all('div', attrs = {'class' : 'thumb'})
    link = link[0].find('a').attrs['href']
    
    
    title_num = link.replace('/webtoon/list?titleId=','')
    link = link.replace('/webtoon/list?titleId=', 'https://comic.naver.com/webtoon/list?titleId=')
    #print(title_num)
    
    image_src = soup.find_all('div', attrs = {'class' : 'thumb'})
    image_src = image_src[0].find('img').attrs['src']
    
    id_list.append(num) ; num += 1
    title_list.append(t)
    day_list.append(day)
    imgSrc_list.append(image_src)
    link_list.append(link)
    title_num_list.append(title_num)
    
    author = soup.find_all('h2')
    author = author[1].find('span', {'class' : 'wrt_nm'}).text[8:] # 7칸의 공백 후 8번 째부터 작가 이름
    author_list.append(author)

    genre = soup.find('span', {'class' : 'genre'}).text
    genre_list.append(genre)

    story = soup.find_all('p')
    story = str(story[3])
    story = story.replace('<p>', '').replace('</p>', '').replace('<br/>', '\n').replace('&lt;', '<').replace('&gt;', '>') # <br>을 개행으로
    story_list.append(story)
    #print(image_src)
    
    driver.back() # 뒤로 가기

C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_9716/2068375710.py:17: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Administrator/Desktop/JavaSpring/Webtoon_recommender/Webtoon_recommender/python_webtoon_crawler/chromedriver.exe')
C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_9716/2068375710.py:23: DeprecationWarning: find_elements_by_class_name is deprecated. Please use find_elements(by=By.CLASS_NAME, value=name) instead
  page = driver.find_elements_by_class_name('title')


In [8]:
total_data = pd.DataFrame()
total_data['id'] = id_list
total_data['title_num'] = title_num_list
total_data['title'] = title_list
total_data['author'] = author_list
total_data['day'] = day_list
total_data['genre'] = genre_list
total_data['story'] = story_list
total_data['link'] = link_list
total_data['img_src'] = imgSrc_list
print(total_data)
total_data.to_csv('naver_webtoon_data.csv', encoding = 'utf-8-sig')

      id title_num      title          author day       genre  \
0      0    758037        참교육       채용택 / 한가람   월     스토리, 액션   
1      1    648419    뷰티풀 군바리        설이 / 윤성원   월    스토리, 드라마   
2      2    783052    퀘스트지상주의        박태준 만화회사   월    스토리, 드라마   
3      3    728750  장씨세가 호위무사       김인호 / 조형근   월  스토리, 무협/사극   
4      4    602910     윈드브레이커             조용석   월    스토리, 스포츠   
..   ...       ...        ...             ...  ..         ...   
567  567    758676    조선여우스캔들  문종호 / 정후 / 차소희   일    스토리, 로맨스   
568  568    799213       어떤소란              케첩   일    스토리, 로맨스   
569  569    773524    거래하실래요?        99C / 백도   일    스토리, 로맨스   
570  570    804055      아포크리파        은성 / 두만식   일    스토리, 스릴러   
571  571    804157  첫날밤만 세 번째       나라나라 / 갓녀   일    스토리, 로맨스   

                                                 story  \
0    무너진 교권을 지키기 위해 교권보호국 소속 나화진의 참교육이 시작된다!\n<부활남>...   
1                       '여자도 군대에 간다면?'본격 여자도 군대 가는 만화!   
2    [외모지상주의], [싸움독학], [인생존망]과 세계관을 공유하는 작품!\

In [22]:
#이미지 링크로 이미지파일 저장(name = title_num)
basic_path = "C:/Users/Administrator/Desktop/JavaSpring/Webtoon_recommender/Webtoon_recommender/src/main/resources/static/img"
    
def save_file(title_num):
    for i in range(len(title_num_list)):
        f = open(basic_path + '/' + str(title_num[i]) + ".jpg", 'wb')
        img_file = urllib.request.urlopen(imgSrc_list[i])
        f.write(img_file.read())
        f.close

In [23]:
save_file(title_num_list)